### Import Libaries  

In [8]:
import math
import pandas as pd
from typing import List, Dict, Any

### Helper Function

In [9]:
def combine_results(*args, n: int) -> Any:
    """
    O(log n) combine function.
    Combines dengue risk assessments from 3 different analysis methods.
    """
    global total_operations
    
    num_ops = max(1, int(math.log2(n))) if n > 1 else 1
    total_operations += num_ops
    
    if all(isinstance(x, dict) for x in args):
        
        current_risk = args[0]
        historical_risk = args[1]
        neighbor_risk = args[2]
        
        combined_risk = (
            current_risk.get('risk_score', 0) * 0.5 +      
            historical_risk.get('risk_score', 0) * 0.3 +   
            neighbor_risk.get('risk_score', 0) * 0.2      
        )
        
        all_zones = []
        for result in args:
            if 'zones' in result:
                all_zones.extend(result['zones'])

        unique = {z['name']: z for z in all_zones}
        all_zones = list(unique.values())

        all_zones.sort(key=lambda z: z['risk'], reverse=True)
        top_zones = all_zones[:3]

        return {
            'risk_score': round(combined_risk, 3),
            'zones': top_zones,
            'alert_level': 'HIGH' if combined_risk > 0.7 else 'MEDIUM' if combined_risk > 0.4 else 'LOW'
        }
    
    return None


### Recursive Engine
T(n) = 3T(n/2) + log n

In [10]:
def recursive_engine(arr: List[Any], depth: int = 0, label: str = "ROOT") -> Any:
    """
    divide-and-conquer recursive engine
    with complexity T(n) = 3T(n/2) + O(log n)
    
    analyzes dengue risk by dividing Singapore into geographic segments.
    """
    global call_counter
    call_counter += 1
    
    n = len(arr)

    # Base case 
    if n == 1:
        zone = arr[0]
        # 1 Current outbreak risk
        current_risk = (
            zone['active_cases'] / 100 * 0.5 +
            zone['breeding_sites'] / 50 * 0.3 +
            zone['population_density'] / 10000 * 0.2
        )

        # 2 Historical seasonal risk
        historical_risk = zone['historical_avg_cases'] / 100

        # 3 Neighbor cross-contamination risk
        neighbor_risk = zone['neighbor_risk_index']

        base_risk = (
            current_risk * 0.5 +
            historical_risk * 0.3 +
            neighbor_risk * 0.2
        )

        return {
            'risk_score': base_risk,
            'zones': [{
                'name': zone['zone_name'],
                'risk': base_risk,
                'cases': zone['active_cases'],
                'current_risk': current_risk,
                'historical_risk': historical_risk,
                'neighbor_risk': neighbor_risk
            }]
        }


    # Divide into two halfs n/2
    mid = n // 2
    first_half = arr[:mid] 
    second_half = arr[mid:]  

    # 3 recursive calls
    # Call 1: Analyze current outbreak patterns in first region
    result1 = recursive_engine(first_half, depth + 1, f"{label}.Current")
    
    # Call 2: Analyze second region for spreading patterns
    result2 = recursive_engine(second_half, depth + 1, f"{label}.Spread")
    
    # Call 3: Re-analyze first region for historical/seasonal patterns
    result3 = recursive_engine(first_half, depth + 1, f"{label}.Historical")

    # Combine step - merge 3 different risk assessments
    combined = combine_results(result1, result2, result3, n=n)
    
    return combined

### Empirical Complexity Tester

In [11]:
def run_empirical_tests(generator_fn, sizes: list) -> pd.DataFrame:
    """
    Runs tests on the generic recursive engine.
    generator_fn(size) should return a list or array for the test.
    """
    global call_counter, total_operations
    results = []

    for n in sizes:
        call_counter = 0
        total_operations = 0

        arr = generator_fn(n)
        recursive_engine(arr)

        theoretical = n ** math.log2(3)
        results.append({
            "n": n,
            "calls": call_counter,
            "operations": total_operations,
            "theoretical": round(theoretical, 2),
            "ratio": round(total_operations / theoretical, 3)
        })

    return pd.DataFrame(results)


## Solving Recurrence 

### Substitution Method 

Time complexity: T(n) = 3T(n / 2) + lgn
Assume 
T(n) ≤ cn ^ log 3 for some constant c

T(n) = 3[ T(n/2)] + lgn 
     = 3T(n/2) + logn ≤ 3c(n/2)^log3 + lgn
     = cn^log3 + lgn.
     
For large n, logn is smaller than n^log3, so T(n) = O(n^log3)

### Recursion Tree Method 




In [12]:
def print_recursion_tree_table(n, max_level=5):
    print(f"{'Level':<8}{'Number of Nodes':<20}{'Cost per Node':<20}{'Total Cost'}")
    print("-" * 70)

    for k in range(max_level):
        num_nodes = 3 ** k
        cost_per_node = f"lg(n/{2**k})"
        total_cost = f"{num_nodes} * lg(n/{2**k})"

        print(f"{k:<8}{num_nodes:<20}{cost_per_node:<20}{total_cost}")


n = 1024
print_recursion_tree_table(n, max_level=6)


Level   Number of Nodes     Cost per Node       Total Cost
----------------------------------------------------------------------
0       1                   lg(n/1)             1 * lg(n/1)
1       3                   lg(n/2)             3 * lg(n/2)
2       9                   lg(n/4)             9 * lg(n/4)
3       27                  lg(n/8)             27 * lg(n/8)
4       81                  lg(n/16)            81 * lg(n/16)
5       243                 lg(n/32)            243 * lg(n/32)


Number of Node = 3^k 

Cost per Node = lg( n/2^k )

Total cost  = 3^k x lg(n/2^k)



Recurrsion depth: logn 

Growth Proportion: n ^ log 3

Therefore T(n) = O(n ^ log3)

### Master Method 

T(n) = 3[ T(n/2) ] + lgn 

a = 3
b = 2
f(n) = lgn 

Since: 
lgn = O(n^c) for c < log3

n^log3 grows faster than any n^c with c < log3 since logn = O(n^c) for every c > 0. f(n) is polynomially smaller than n^log3.

This falls under Case 1 of Master Theorem 
Therefore: 
T(n) = n^log3

## Singapore Context Problem



### Load Dengue Data 


In [13]:
def load_dengue_csv(filepath: str) -> List[Dict[str, Any]]:
    """
    Loads dengue zone data from CSV into list of dictionaries.
    """
    df = pd.read_csv(filepath)
    return df.to_dict(orient="records")

def dengue_zone_generator_from_csv(filepath: str, n: int) -> List[Dict[str, Any]]:
    """
    Converts CSV-based dengue data into zone risk assessments.
    Each zone represents a geographic area in Singapore.
    """
    zones = load_dengue_csv(filepath)[:n]
    return zones



### Dengue Main Execution

Background: NEA monitors dengue clusters across Singapore. 
This algorithm analyzes geographic zones using 3 risk factors to discover hotspots in singapore:
 1. Current active outbreak cases
 2. Historical seasonal patterns
 3. Cross-contamination from neighboring zones

In [14]:
print("\n" + "="*70)
print("SINGAPORE DENGUE HOTSPOT RISK ANALYSIS")
print("="*70)

call_counter = 0
total_operations = 0

csv_file = "dengue_zones.csv"

# Analyze 8 geographic zones in this example
print("\nAnalyzing geographic zones in Singapore...")
zone_data = dengue_zone_generator_from_csv(csv_file, 8)

print("\nZone Data:")
for zone in zone_data:
    print(f"  {zone['zone_name']}: {zone['active_cases']} cases, "
          f"{zone['breeding_sites']} breeding sites, "
          f"density {zone['population_density']}/km²")

dengue_result = recursive_engine(zone_data)

print("\n" + "="*70)
print("DENGUE RISK ASSESSMENT RESULTS:")
print("="*70)
print(f"Overall Risk Score: {dengue_result['risk_score']:.3f}")
print(f"Alert Level: {dengue_result['alert_level']}")

print("\nTop 3 High-Risk Zones:")
for i, zone in enumerate(dengue_result['zones'][:3], 1):
    print(f"  {i}. {zone['name']}: Risk {zone['risk']:.3f}, {zone['cases']} active cases")

print(f"\nTotal Recursive Calls: {call_counter}")
print(f"Total Operations: {total_operations}")

test_sizes_dengue = [2, 4, 8, 16]
dengue_empirical_results = run_empirical_tests(
    lambda n: dengue_zone_generator_from_csv(csv_file, n),
    test_sizes_dengue
)

print("\n" + "="*70)
print("EMPIRICAL COMPLEXITY VERIFICATION")
print("="*70)
print(dengue_empirical_results.to_string(index=False))
print("\nComplexity confirmed: T(n) = Θ(n^(log₂ 3)) ≈ Θ(n^1.585)")

    


SINGAPORE DENGUE HOTSPOT RISK ANALYSIS

Analyzing geographic zones in Singapore...

Zone Data:
  Ang Mo Kio: 45 cases, 28 breeding sites, density 8500/km²
  Bedok: 67 cases, 42 breeding sites, density 9200/km²
  Bishan: 38 cases, 22 breeding sites, density 7800/km²
  Bukit Batok: 52 cases, 31 breeding sites, density 8100/km²
  Bukit Merah: 71 cases, 48 breeding sites, density 9800/km²
  Bukit Panjang: 41 cases, 25 breeding sites, density 7600/km²
  Bukit Timah: 29 cases, 15 breeding sites, density 6200/km²
  Choa Chu Kang: 48 cases, 29 breeding sites, density 7900/km²

DENGUE RISK ASSESSMENT RESULTS:
Overall Risk Score: 0.540
Alert Level: MEDIUM

Top 3 High-Risk Zones:
  1. Bukit Merah: Risk 0.765, 71 active cases
  2. Bedok: Risk 0.693, 67 active cases
  3. Bukit Batok: Risk 0.558, 52 active cases

Total Recursive Calls: 40
Total Operations: 18

EMPIRICAL COMPLEXITY VERIFICATION
 n  calls  operations  theoretical  ratio
 2      4           1          3.0  0.333
 4     13           5 